In [2]:
import fmp # fmp.py contains all helper functions working with FMP API for financial data
import pandas as pd

# Dow Jones: select stocks which have positive 1Y, 2Y and 3Y momentum

In [4]:
# Retrieve symbols and financials

sp500_tickers = fmp.sp500_symbols()
dow_tickers = fmp.dow_symbols()

# sp500_financials = dataSrc.financials(sp500_tickers)

df = fmp.extract_financials(dow_tickers)

# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))

df = df[conditions]

print(df)

# Compute historical momentum

m12_momentums = []
m24_momentums = []
m36_momentums = []
count = 0
for symbol in df["Symbol"]:
    count += 1
    m12_momentums.append(fmp.calculate_hist_momentum(symbol, 252))
    m24_momentums.append(fmp.calculate_hist_momentum(symbol, 504))
    m36_momentums.append(fmp.calculate_hist_momentum(symbol, 756))

df["m12_momentum"] = m12_momentums
df["m24_momentum"] = m24_momentums
df["m36_momentum"] = m36_momentums

positive_mom_stocks = df[(df['m12_momentum'] > 0) & (df['m24_momentum'] > 0)
                         & (df['m36_momentum'] > 0)]
positive_mom_stocks

CRM: 1/30
WBA: 2/30
V: 3/30
NKE: 4/30
UNH: 5/30
TRV: 6/30
VZ: 7/30
INTC: 8/30
WMT: 9/30
JNJ: 10/30
DIS: 11/30
MCD: 12/30
JPM: 13/30
CAT: 14/30
BA: 15/30
AMGN: 16/30
DOW: 17/30
AAPL: 18/30
GS: 19/30
CSCO: 20/30
MSFT: 21/30
HD: 22/30
PG: 23/30
MRK: 24/30
IBM: 25/30
HON: 26/30
KO: 27/30
CVX: 28/30
AXP: 29/30
MMM: 30/30
   Symbol                            Name Exchange                  Sector  \
1     WBA  Walgreens Boots Alliance, Inc.   NASDAQ              Healthcare   
3     NKE                      NIKE, Inc.     NYSE       Consumer Cyclical   
5     TRV   The Travelers Companies, Inc.     NYSE      Financial Services   
6      VZ     Verizon Communications Inc.     NYSE  Communication Services   
7    INTC               Intel Corporation   NASDAQ              Technology   
9     JNJ               Johnson & Johnson     NYSE              Healthcare   
19   CSCO             Cisco Systems, Inc.   NASDAQ              Technology   
20   MSFT           Microsoft Corporation   NASDAQ        

,Symbol,Name,Exchange,Sector,Industry,MarketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,m12_momentum,m24_momentum,m36_momentum
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,258.979332,0.595383,0.514070,0.448202,0.043478,0.006724,1.100,0.716614,0.386356,0.865975,0.979789
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,39.010726,0.481540,0.132920,0.249376,0.280277,0.022128,3.460,0.317382,0.378836,0.054349,0.222996
7,INTC,Intel Corporation,NASDAQ,Technology,Semiconductors,218.428883,0.743302,0.231499,0.555936,0.506024,0.025501,1.373,0.374296,0.092532,0.015657,0.198308
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,439.545201,0.338716,0.270051,0.669963,0.012766,0.024795,4.140,0.348304,0.129855,0.280150,0.205473
19,CSCO,"Cisco Systems, Inc.",NASDAQ,Technology,Communication Equipment,244.164968,0.247370,0.266846,0.640210,0.048827,0.025220,1.460,0.433121,0.451605,0.159423,0.230916
20,MSFT,Microsoft Corporation,NASDAQ,Technology,Software—Infrastructure,2222.228242,1.328849,0.462317,0.689258,0.064902,0.007575,2.240,0.553834,0.449346,1.150305,1.658306
21,HD,"The Home Depot, Inc.",NYSE,Consumer Cyclical,Home Improvement Retail,350.326751,0.813120,7.047047,0.337181,0.173127,0.019431,6.450,1.387776,0.201281,0.419803,0.552256


# Calculate stocks' discount rate from 52W high

In [ ]:
watchlist = positive_mom_stocks[["symbol", "name"]].copy()

# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []
count = 0

for symbol in watchlist["symbol"]:
    count += 1
    print(f"{symbol}: {count}/{len(watchlist['symbol'])}")
    currentPrice = fmp.get_current_price(symbol)
    high = fmp.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high * 100

    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)

watchlist["Current Price"] = currentPrices
watchlist["52W_High"] = highs
watchlist["Discount (%)"] = discounts

watchlist = watchlist.sort_values(by="Discount (%)")
watchlist

# Consider buying if discounted more than 15%

In [ ]:
buylist = watchlist[watchlist['Discount (%)'] < -15]
buylist

# S&P500

In [ ]:
# Retrieve symbols and financials

sp500_tickers = fmp.sp500_symbols()

df = fmp.extract_financials(sp500_tickers)

# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))

df = df[conditions]

# Compute historical momentum

m12_momentums = []
m24_momentums = []
m36_momentums = []
count = 0
for symbol in df["symbol"]:
    count += 1
    m12_momentums.append(fmp.calculate_hist_momentum(symbol, 252))
    m24_momentums.append(fmp.calculate_hist_momentum(symbol, 504))
    m36_momentums.append(fmp.calculate_hist_momentum(symbol, 756))

df["m12_momentum"] = m12_momentums
df["m24_momentum"] = m24_momentums
df["m36_momentum"] = m36_momentums

positive_mom_stocks = df[(df['m12_momentum'] > 0) & (df['m24_momentum'] > 0)
                         & (df['m36_momentum'] > 0)]
positive_mom_stocks

In [ ]:
watchlist = positive_mom_stocks[["symbol", "name"]].copy()

# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []
count = 0

for symbol in watchlist["symbol"]:
    count += 1
    print(f"{symbol}: {count}/{len(watchlist['symbol'])}")
    currentPrice = fmp.get_current_price(symbol)
    high = fmp.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high * 100

    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)

watchlist["Current Price"] = currentPrices
watchlist["52W_High"] = highs
watchlist["Discount (%)"] = discounts

watchlist = watchlist.sort_values(by="Discount (%)")
watchlist

In [ ]:
buylist = watchlist[watchlist['Discount (%)'] < -15]
buylist

In [ ]:
#!jupyter nbconvert --to script --no-prompt analysis.ipynb